# 1. Introduction

## 1.1 Business Understanding

## 1.2 Data understanding

The dataset used for this anaysis was downloaded from [DrivenData](https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/data/). The data was was acquired from [Taarifa](https://taarifa.org/) and [Tanzanian Ministry of Water](https://www.maji.go.tz/). The data contains information about wells in Tanzania. The data has *** columns. The data has the following set of information about the water pumps:
- amount_tsh - Total static head (amount water available to waterpoint)
- date_recorded - The date the row was entered
- funder - Who funded the well
- gps_height - Altitude of the well
- installer - Organization that installed the well
- longitude - GPS coordinate
- latitude - GPS coordinate
- wpt_name - Name of the waterpoint if there is one
- num_private - Private use or not
- basin - Geographic water basin
- subvillage - Geographic location
- region - Geographic location
- region_code - Geographic location (coded)
- district_code - Geographic location (coded)
- lga - Geographic location
- ward - Geographic location
- population - Population around the well
- public_meeting - True/False
- recorded_by - Group entering this row of data
- scheme_management - Who operates the waterpoint
- scheme_name - Who operates the waterpoint
- permit - If the waterpoint is permitted
- construction_year - Year the waterpoint was constructed
- extraction_type - The kind of extraction the waterpoint uses
- extraction_type_group - The kind of extraction the waterpoint uses
- extraction_type_class - The kind of extraction the waterpoint uses
- management - How the waterpoint is managed
- management_group - How the waterpoint is managed
- payment - What the water costs
- payment_type - What the water costs
- water_quality - The quality of the water
- quality_group - The quality of the water
- quantity - The quantity of water
- quantity_group - The quantity of water
- source - The source of the water
- source_type - The source of the water
- source_class - The source of the water
- waterpoint_type - The kind of waterpoint
- waterpoint_type_group - The kind of waterpoint


# 2. Data wrangling

## 2.1 Loading libraries and datasets

In [1]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

# import warnings
# warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split

In [2]:
# expanding the number of visible columns

pd.set_option('display.max_columns',999)

In [3]:
# loading the data

data = pd.read_csv('Data/Raw/Training set values.csv')
data_labels = pd.read_csv('Data/Raw/Training set labels.csv')

In [5]:
# merging the dataframes 

df = pd.merge(left = data, right = data_labels, on = 'id')

In [6]:
# previewing the complete dataframe

df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


## 2.2 Descriptive statistics

In [8]:
# previewing the shape of the data

print(f'The data has {df.shape[0]} rows and {df.shape[1]} columns')

The data has 59400 rows and 41 columns


In [11]:
# getting the statistics of the numerical columns

df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,59400.0,37115.131768,21453.128371,0.00000,18519.750000,37061.500000,55656.500000,7.424700e+04
amount_tsh,59400.0,317.650385,2997.574558,0.00000,0.000000,0.000000,20.000000,3.500000e+05
gps_height,59400.0,668.297239,693.116350,-90.00000,0.000000,369.000000,1319.250000,2.770000e+03
longitude,59400.0,34.077427,6.567432,0.00000,33.090347,34.908743,37.178387,4.034519e+01
latitude,59400.0,-5.706033,2.946019,-11.64944,-8.540621,-5.021597,-3.326156,-2.000000e-08
num_private,59400.0,0.474141,12.236230,0.00000,0.000000,0.000000,0.000000,1.776000e+03
region_code,59400.0,15.297003,17.587406,1.00000,5.000000,12.000000,17.000000,9.900000e+01
district_code,59400.0,5.629747,9.633649,0.00000,2.000000,3.000000,5.000000,8.000000e+01
population,59400.0,179.909983,471.482176,0.00000,0.000000,25.000000,215.000000,3.050000e+04
construction_year,59400.0,1300.652475,951.620547,0.00000,0.000000,1986.000000,2004.000000,2.013000e+03


## 2.3 Data cleaning

Before and analysis and preprocessing is done on the data, the data needs to be cleaned in case of any inconsistencies. The first aspect that needs to be checked is whether the data has any null values.

In [12]:
# checking for null values in the dataset

df.isna().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [28]:
# calculating the percentage of missing values

null_values = pd.DataFrame(df.isna().sum()/len(df)*100).rename(columns = {0:'Null Percentage'})
null_values = null_values[(null_values['Null Percentage'] > 0)]
null_values.sort_values(by = 'Null Percentage', ascending = False)

,Null Percentage
scheme_name,47.417508
scheme_management,6.526936
installer,6.153199
funder,6.119529
public_meeting,5.612795
permit,5.144781
subvillage,0.624579


`scheme_name` column has 47% of it's data missing. This percentage is too high, hence the column needs to be dropped.

In [42]:
# dropping the column 'scheme_name'

df.drop(columns = 'scheme_name', inplace = True)

In [43]:
df[['scheme_management', 'installer', 'funder', 'public_meeting','permit','subvillage']].head()

,scheme_management,installer,funder,public_meeting,permit,subvillage
0,VWC,Roman,Roman,True,False,Mnyusi B
1,Other,GRUMETI,Grumeti,NaN,True,Nyamara
2,VWC,World vision,Lottery Club,True,True,Majengo
3,VWC,UNICEF,Unicef,True,True,Mahakamani
4,NaN,Artisan,Action In A,True,True,Kyanyamisa


No clear pattern could be found for why the data was missing. So the missing values will be imputed in sucha a way that the missing values will still be missing

In [47]:
# imputing the missing values

df.fillna(value = 'MISSING', inplace = True)

All the  missing values have been handled, and the data is now complete

The next aspect to look into in whether the data has any duplicates.

In [54]:
# checking for duplicates

print(f'The data has {df.duplicated().sum()} duplicates')

The data has 0 duplicates


The next aspect to look into is the uniformity of the data. This check will include looking at the data types of the different columns and ensuring they are are correct.

In [53]:
# checking the column datatypes

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 59400 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              59400 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59400 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

The columns `date_recorded` has the data types object. This needs to be converted to a date-type data type.

In [58]:
# converting the column's data type

df['date_recorded'] = pd.to_datetime(df['date_recorded'])

# 3. Exploratory Data Analysis(EDA)

In [59]:
!pip install geopandas

    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\w.selen\Anaconda3\envs\learn-env\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\w.selen\\AppData\\Local\\Temp\\pip-install-azpz6izw\\fiona\\setup.py'"'"'; __file__='"'"'C:\\Users\\w.selen\\AppData\\Local\\Temp\\pip-install-azpz6izw\\fiona\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\w.selen\AppData\Local\Temp\pip-pip-egg-info-1iqw7p5r'
         cwd: C:\Users\w.selen\AppData\Local\Temp\pip-install-azpz6izw\fiona\
    Complete output (1 lines):
    A GDAL API version must be specified. Provide a path to gdal-config using a GDAL_CONFIG environment variable or use a GDAL_VERSION environment variable.
    ----------------------------------------
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the 

# 4. Modelling

# 5. Evaluation